In [ ]:
import os
import datetime
import geopandas as gpd
import pandas as pd
import json
import shapely.ops

import sentinelhub
import boto3

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import config

import cdseutils.utils as utils
import cdseutils.mydataclasses as mydataclasses
import cdseutils.constants as constants
import cdseutils.sentinel2 as sentinel2

import mysecrets
import s2l1c_via_s3

In [ ]:
cdse_creds = mysecrets.PROFILES['nikhilsasirajan@gmail.com']

In [ ]:
utils.cdse_credentials_to_json(cdse_creds=cdse_creds, filepath='../data/cdse_credentials.json')

In [ ]:
_cdse_creds = utils.cdse_credentials_from_json(filepath='../data/cdse_credentials.json')

In [ ]:
_cdse_creds.sh_creds == cdse_creds.sh_creds, _cdse_creds.s3_creds == cdse_creds.s3_creds

In [ ]:
year = 2021

startdate = datetime.datetime(year, 3, 1)
enddate = datetime.datetime(year + 1, 1, 31)

In [ ]:
(enddate - startdate).days

In [ ]:
catalog_filepath = '../data/eth_bb_east_s2l1c_catalog.geojson'
# roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2019_North_Bounding_Box.shp'
# roi_filepath = '../data/test_eth_east_bb_s2grids.geojson',
eth_s2grids_filepath = '../../ethiopia/data/ethiopia_s2_grids_esa_stats.geojson'

In [ ]:
# catalog_gdf = gpd.read_file(catalog_filepath)

In [ ]:
eth_s2grids_gdf = gpd.read_file(eth_s2grids_filepath)

In [ ]:
s2grid_of_interest = '165bca4'

In [ ]:
selected_s2grid_gdf = eth_s2grids_gdf[
    eth_s2grids_gdf['id'] == s2grid_of_interest
][['id','geometry']]
selected_s2grid_gdf

In [ ]:
selected_s2grid_gdf.to_file('../data/s2grid_165bca4.geojson')

In [ ]:
# shapes_gdf = gpd.read_file(roi_filepath)
# bboxes = utils.get_bboxes(shapes_gdf=shapes_gdf)
bboxes = utils.get_bboxes(shapes_gdf=selected_s2grid_gdf)
catalog_gdf, results = utils.fetch_catalog(
    bboxes = bboxes,
    sh_creds = cdse_creds.sh_creds,
    collection = sentinelhub.DataCollection.SENTINEL2_L1C,
    startdate = startdate,
    enddate = enddate,
    cache_folderpath = config.FOLDERPATH_FETCH_CATALOG_CACHE,
)

In [ ]:
len(results)

In [ ]:
results

In [ ]:
s3_creds = cdse_creds.s3_creds
s3url = 's3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/03/02/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652.SAFE/'

In [ ]:
s3 = boto3.resource(
    's3',
    endpoint_url = s3_creds.endpoint_url,
    aws_access_key_id = s3_creds.s3_access_key,
    aws_secret_access_key = s3_creds.s3_secret_key,
    region_name = s3_creds.region_name,
)

root_s3path = utils.s3url_to_s3path(s3url=s3url)

all_files = s3.Bucket(root_s3path.bucket).objects.filter(Prefix=root_s3path.prefix)

In [ ]:
all_files = list(all_files)

In [ ]:
xml_s3paths = [mydataclasses.S3Path(bucket=file.bucket_name, prefix=file.key) for file in all_files if file.key.endswith('.xml')]

In [ ]:
xml_s3paths

In [ ]:
for s3path in xml_s3paths:
    utils.download_s3_file(
        s3_creds = cdse_creds.s3_creds,
        s3path = s3path,
        download_folderpath = '../data/xml_files/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652'
    )

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
root = ET.parse('../data/xml_files/S2A_MSIL1C_20210302T074831_N0500_R135_T36PZU_20230523T123652/MTD_TL.xml').getroot()

In [ ]:
root

In [ ]:
element_mean_sun_angle = root.findall('.//Mean_Sun_Angle/AZIMUTH_ANGLE')[0]
element_mean_sun_angle.text, element_mean_sun_angle.attrib['unit']

In [ ]:
root.findall('.//Mean_Sun_Angle/AZIMUTH_ANGLE')[0].attrib['unit']

In [ ]:
{x['properties']['proj:epsg'] for x in results}

In [ ]:
{x['properties']['proj:epsg'] for x in results}

In [ ]:
catalog_gdf[:2]

In [ ]:
successful_download_count, total_download_count \
= s2l1c_via_s3.download_sentinel2_l1c_tiles(
    cdse_creds = cdse_creds,
    catalog_gdf = catalog_gdf[:2],
    sentinel2_local_catalog_filepath = config.FILEPATH_SENTINEL2_LOCAL_CATALOG,
    catalog_save_folderpath = config.FOLDERPATH_FETCH_CATALOG_CACHE,
    root_download_folderpath = config.FOLDERPATH_SATELLITE,
    roi_filepath = roi_filepath,
    startdate = startdate,
    enddate = enddate,
    upper_limit_for_number_of_tiles = 50,
    # chunksize_for_download_and_update_catalog = 100,
)

In [ ]:
local_catalog_gdf = gpd.read_file(config.FILEPATH_SENTINEL2_LOCAL_CATALOG)

In [ ]:
local_catalog_gdf